In [2]:
import os
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
import pyproj
from shapely.geometry import Point
from pathlib import Path
import re
from rasterio.mask import mask
import json
import datetime
from os import listdir
from rasterstats import zonal_stats

/opt/anaconda3/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [95]:
# #construct scene name
# prefix = 'LC08_L2SP'
# path_row = ['022031', '023031']
# y = list(map(str,list(range(2021,2012, -1))))
# acq_m = ['05', '06', '07', '08', '09']
# pro_m = ['0'+str(n) if len(str(n))==1 else str(n) \
#      for n in np.arange(1, 13, 1)]
# d = ['0'+str(n) if len(str(n))==1 else str(n) \
#      for n in np.arange(1, 32, 1)]
# suffix = '02_T1'

# all_scenes = []
# for p_r in path_row:
#     for year in y:
#         for month in acq_m:
#             for day in d:
#                 for year2 in y:
#                     for month2 in pro_m:
#                         for day2 in d:
#                             acq_date = year+month+day
#                             pro_date = year2+month2+day2
#                             scene = f"{prefix}_{p_r}_{acq_date}_{pro_date}_{suffix}"
#                             all_scenes.append(scene)

In [8]:
SHARED_DATA_FOLDER = Path('sample_data')
all_scenes = listdir(SHARED_DATA_FOLDER)

In [4]:
# SCENE = 'LC08_L2SP_022031_20140103_20200912_02_T1'
# scene_path = SHARED_DATA_FOLDER/SCENE
# 'LC08' in SCENE

In [26]:
#Need this on all cores
def read_spatial(path, espg_code):
    '''
    Function to read spatial data and converts to ESPG 3435
    Input: path to the data file
            epgs_code (string): epsg code as a string
    Output: a gpd object 
    '''
    epsg = "EPSG:" + espg_code
    file=gpd.read_file(path)
    file=file.to_crs(epsg)
    print(path, file.crs)
    return file

def compute_zonal_stats(path_to_raster, vector, band_name):
    '''
    Inputs: 
        path_to_raster (string): path to raster data
        vector (geopandas df): 
        band_name (string): name of band
    Outputs:
        nparray (1D np array): mean values of the raster data 
                             ordered by com area order
    '''
    col_name = "mean_" + band_name
    sum_stats = zonal_stats(vector, path_to_raster, 
                            # nodata = Nan,
                            stats=["mean"])
    df = pd.DataFrame(sum_stats)
    df = df.rename(columns = {"mean": col_name})
    nparray = np.array(df)
    
    return nparray.flatten()

In [13]:
#Need these on all cores
com_areas = read_spatial("data/com_areas_chi", "32616")
com_areas = com_areas[["area_numbe", "community", "geometry"]]

data/com_areas_chi EPSG:32616


In [13]:
rasterio.open(band_path[0]).read(1)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint16)

In [11]:

band_names = ['SR_B1','SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', \
              'SR_B6', 'SR_B7', 'ST_B10']
band_path = []

for SCENE in all_scenes:
    if 'LC08' not in SCENE:  #filtering out metadata folders
        continue
    scene_path = SHARED_DATA_FOLDER/SCENE
    for band in band_names:
        path = scene_path/"{}_{}.TIF".format(SCENE, band)
        band_path.append(path)
    b1 = compute_zonal_stats(band_path[0], com_areas, "b1")
    b2 = compute_zonal_stats(band_path[1], com_areas, "b2")
    b4 = compute_zonal_stats(band_path[3], com_areas, "b4")
    print(b4)
    b5 = compute_zonal_stats(band_path[4], com_areas, "b5")
    ndvi = np.where((b4+b5)==0, 0, (b5-b4)/(b5+b4))
    print(ndvi)

[[26113.42370027]
 [28930.62407723]
 [23327.95586809]
 [25349.77973922]
 [22103.54740369]
 [21984.40871228]
 [29465.96774194]
 [21875.04937473]
 [26520.65192534]
 [19757.80751085]
 [22721.72591486]
 [20542.55860866]
 [24334.45104799]
 [20559.78314992]
 [21662.99498195]
 [20084.3091777 ]
 [24913.6366701 ]
 [23108.22960085]
 [23112.03343384]
 [22011.38942497]
 [22022.81097197]
 [20336.63332747]
 [19165.05341591]
 [23880.10351468]
 [19598.42719854]
 [24060.14010508]
 [24551.63575449]
 [25781.54499549]
 [22028.04013972]
 [25897.46768965]
 [22871.61515513]
 [24643.00653148]
 [23451.32633087]
 [24956.91803599]
 [22083.86580238]
 [23606.29242882]
 [17153.42993302]
 [19044.72497366]
 [22737.80506569]
 [24148.34945159]
 [23695.96106785]
 [27668.41338174]
 [26667.04449515]
 [23677.18629391]
 [17288.55415925]
 [25651.19139614]
 [25589.74303048]
 [19015.91806901]
 [29529.81504599]
 [32850.27448791]
 [28575.83922117]
 [25634.61964147]
 [32302.81119019]
 [33913.91341245]
 [26378.77443238]
 [25925.86

[[-0.00445241]
 [-0.00999873]
 [ 0.00594629]
 [-0.00269924]
 [ 0.01293625]
 [ 0.0181055 ]
 [-0.01024172]
 [ 0.01368396]
 [-0.00299644]
 [ 0.02632949]
 [ 0.01284477]
 [ 0.03741658]
 [ 0.01085418]
 [ 0.01922249]
 [ 0.01349287]
 [ 0.02500513]
 [ 0.00202125]
 [ 0.00786842]
 [ 0.00284386]
 [ 0.01560948]
 [ 0.00821416]
 [ 0.01655697]
 [ 0.02253363]
 [ 0.00265351]
 [ 0.01799984]
 [ 0.00217435]
 [ 0.00011609]
 [-0.00281835]
 [ 0.00341616]
 [-0.00326541]
 [ 0.01086944]
 [-0.00139988]
 [-0.0035848 ]
 [-0.00276663]
 [ 0.00277711]
 [ 0.01131376]
 [ 0.01074742]
 [-0.00073465]
 [ 0.00668468]
 [ 0.00289355]
 [ 0.00663257]
 [-0.00571739]
 [-0.00177575]
 [ 0.00117274]
 [ 0.0355979 ]
 [-0.00210052]
 [-0.00086585]
 [ 0.02880812]
 [-0.00894908]
 [-0.01583133]
 [-0.01244595]
 [ 0.00034609]
 [-0.01088682]
 [-0.01763464]
 [-0.01025893]
 [-0.00629846]
 [ 0.00713706]
 [ 0.00197058]
 [ 0.0002041 ]
 [-0.00190891]
 [ 0.00258719]
 [-0.01545436]
 [-0.00990348]
 [-0.00066138]
 [ 0.00809406]
 [ 0.00121124]
 [ 0.00236

In [8]:
SHARED_DATA_FOLDER = Path('tif_data_2021/practice')

listdir(SHARED_DATA_FOLDER)

['LC08_L2SP_023031_20210910_20210916_02_T1',
 '.DS_Store',
 'LC08_L2SP_023031_20210825_20210901_02_T1',
 'LC08_L2SP_023031_20210926_20211001_02_T1']

In [34]:
SHARED_DATA_FOLDER = Path('tif_data_2021/practice')
all_scenes = listdir(SHARED_DATA_FOLDER)
band_names = ['SR_B1','SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', \
              'SR_B6', 'SR_B7', 'ST_B10']


def compute_aggregate_scene_data(SCENE):
    '''
    Function that wraps around compute_zonal_stats to compute zonal stats
    for all 8 bands and then aggregate the results of the scene into a single
    matrix 
    
    Input:
        SCENE (string)- path to a folder containing a raster scene
    
    Output: 
        agg_bands (2D np array): a 77 x 9 array that conta
    '''
    
    #Build list of paths for each band in the scene
    scene_path = SHARED_DATA_FOLDER/SCENE
    band_path = []
    for band in band_names:
        path = scene_path/"{}_{}.TIF".format(SCENE, band)
        band_path.append(path)

    #Access all bands
    b1 = compute_zonal_stats(band_path[0], com_areas, "b1")
    b2 = compute_zonal_stats(band_path[1], com_areas, "b2")
    b3 = compute_zonal_stats(band_path[0], com_areas, "b3")
    b4 = compute_zonal_stats(band_path[3], com_areas, "b4")
    b5 = compute_zonal_stats(band_path[4], com_areas, "b5")
    b6 = compute_zonal_stats(band_path[0], com_areas, "b6")
    b7 = compute_zonal_stats(band_path[0], com_areas, "b7")
    b10 = compute_zonal_stats(band_path[0], com_areas, "b10")
    
    agg_bands = np.array([b1, b2, b3, b3, b5, b6, b7, b10]).T
    return agg_bands
    
    
    
    

In [37]:
r = compute_aggregate_scene_data('LC08_L2SP_023031_20210910_20210916_02_T1')

tif_data_2021/practice/LC08_L2SP_023031_20210910_20210916_02_T1/LC08_L2SP_023031_20210910_20210916_02_T1_SR_B1.TIF


In [38]:
r.shape

(77, 8)

In [ ]:
#Compute attributes from bands:
    ndvi = np.where((b4+b5)==0, 0, (b5-b4)/(b5+b4))
    
    ndsi =np.where((b3+b6)==0, 0, (b3-b6)/(b3+b6))
    
    albedo = ((0.356*b1)+(0.1310*b2)+(0.373*b3)+(0.085*b4)+(0.072*b5)-0.0018)/1.016

    awei = 4*(b3-b6)-(0.25*b5 + 2.75*b6)
    
    ndbi = np.where((b6+b5)==0, 0, (b6-b5)/(b6+b5))
    
    eta = (2*(b5**2-b4**2) + 1.5*b5 + 0.5*b4) / (b5+b4+0.5)
    
    gemi = eta*(1-0.25*eta) - ((b4-0.125)/(1-b4))
    

In [20]:
# raster = rasterio.open(b1_path)
# array_dtype = np.dtype('uint16')
# i,j = np.indices(b1.shape, array_dtype)
# longitude, latitude = raster.transform * (j, i)

In [5]:
# pattern = '^(?:[^_]+_){3}([^_ ]+)'
# period_str= re.findall(pattern, SCENE)[0]

In [8]:
ndsi =np.where((b3+b6)==0, 0, (b3-b6)/(b3+b6))
albedo = ((0.356*b1)+(0.1310*b2)+(0.373*b3)+(0.085*b4)+(0.072*b5)-0.0018)/1.016
awei = 4*(b3-b6)-(0.25*b5 + 2.75*b6)
ndvi = np.where((b4+b5)==0, 0, (b5-b4)/(b5+b4))
ndbi = np.where((b6+b5)==0, 0, (b6-b5)/(b6+b5))
eta = (2*(b5**2-b4**2) + 1.5*b5 + 0.5*b4) / (b5+b4+0.5)
gemi = eta*(1-0.25*eta) - ((b4-0.125)/(1-b4))

<ipython-input-8-45c57a3d1d06>:1: RuntimeWarning: divide by zero encountered in true_divide
  ndsi =np.where((b3+b6)==0, 0, (b3-b6)/(b3+b6))
<ipython-input-8-45c57a3d1d06>:1: RuntimeWarning: invalid value encountered in true_divide
  ndsi =np.where((b3+b6)==0, 0, (b3-b6)/(b3+b6))
<ipython-input-8-45c57a3d1d06>:4: RuntimeWarning: divide by zero encountered in true_divide
  ndvi = np.where((b4+b5)==0, 0, (b5-b4)/(b5+b4))
<ipython-input-8-45c57a3d1d06>:4: RuntimeWarning: invalid value encountered in true_divide
  ndvi = np.where((b4+b5)==0, 0, (b5-b4)/(b5+b4))
<ipython-input-8-45c57a3d1d06>:5: RuntimeWarning: divide by zero encountered in true_divide
  ndbi = np.where((b6+b5)==0, 0, (b6-b5)/(b6+b5))
<ipython-input-8-45c57a3d1d06>:5: RuntimeWarning: invalid value encountered in true_divide
  ndbi = np.where((b6+b5)==0, 0, (b6-b5)/(b6+b5))
<ipython-input-8-45c57a3d1d06>:7: RuntimeWarning: divide by zero encountered in true_divide
  gemi = eta*(1-0.25*eta) - ((b4-0.125)/(1-b4))


In [9]:
columns = ['ndsi', 'albedo', 'awei', 'ndbi', 'ndvi', 'gemi', 'latitude', 'longitude', 'lst']
data = {}
features = [ndsi, albedo, awei, ndbi, ndvi, gemi, latitude, longitude]
for i, feature in enumerate(features):
    key = columns[i]
    data[key] = np.ndarray.flatten(feature)

In [11]:
landsat = pd.DataFrame(data)
landsat['period'] = period_str
landsat['period'] = pd.to_datetime(landsat['period'],yearfirst=True, format ='%Y%m%d')

In [84]:
landsat.to_csv("sample_df")